| <img src="dispersed.png" alt="dispersed.png" width="300"/> | <img src="oblate.png" alt="oblate.png" width="300"/> |<img src="final.png" alt="final.png" width="300"/> |
|:-:|:-:|:-:|
|Dispersed state, `0 ns`| Intermediate oblate micelle, `0.15 ns`| Spherical micelle, `0.3 ns`|

### Example: SDS
We setup a simulation box of randomly dispersed charged SDS surfactants with counter-ions in water, run a short simulation, and examine the resulting trajectory.

In [10]:
import h5py
import numpy as np
from scipy.spatial.transform import Rotation as R


def setup_system(box_size, n_molecules, n_solvent, out_path):
    """Setup a simple system of SDS, counter-ions, and water
    
    Parameters
    ----------
    box_size : list of float
        Simulation box size in nm.
    n_molecules : int
        Total number of SDS molecules. Also the number of 
        positively charged counter-ions.
    n_solvent : int
        Total number of waters added to the simulation box.
    out_path : str
        Path of the created structure file.
    """
    sds_single_positions = np.array([
        [0.0, 0.0, 0.0],
        [0.5, 0.0, 0.0],
        [1.0, 0.0, 0.0],
        [1.5, 0.0, 0.0],
    ], dtype=np.float32)
    sds_single_bonds = np.array([
        [1, np.nan],
        [0, 2],
        [1, 3],
        [2, np.nan],
    ], np.float32)
    sds_single_charges = np.array([-1, 0, 0, 0], dtype=np.int32)
    sds_single_types = np.array([0, 1, 1, 1,], dtype=np.int32)
    sds_single_names = np.array(["S", "C", "C", "C"])
    
    box_size = np.asarray(box_size, dtype=np.float32)
    chain_length = 4
    n_particles = n_molecules * (chain_length + 1) + n_solvent
    sds_molecules = np.repeat(np.arange(n_molecules), chain_length)
    sds_bonds = (
        np.tile(sds_single_bonds, (n_molecules,1)) 
        + chain_length * sds_molecules[:, None]
    )
    bonds = np.concatenate(
        (
            sds_bonds, 
            np.full(shape=(n_molecules, 2), fill_value=np.nan, dtype=np.float32),
            np.full(shape=(n_solvent, 2), fill_value=np.nan, dtype=np.float32),
        ),
    )
    bonds[np.where(np.isnan(bonds))] = -1
    bonds = bonds.astype(np.int32) 
    molecules_solvent = np.arange(n_solvent) + 2 * n_molecules
    molecules_counter_ions = np.arange(n_molecules) + n_molecules
    molecules = np.concatenate(
        (sds_molecules, molecules_counter_ions, molecules_solvent)
    )  
    names = np.concatenate(
        (
            np.tile(sds_single_names, n_molecules),
            np.tile(np.array(["Na"]), n_molecules),
            np.tile(np.array(["W"]), n_solvent),
        ),
    )
    types = np.concatenate(
        (
            np.tile(sds_single_types, n_molecules),
            np.tile(np.array([2], dtype=int), n_molecules),
            np.tile(np.array([3], dtype=int), n_solvent),
        ),
    )
    charges = np.concatenate(
        (
            np.tile(sds_single_charges, n_molecules),
            np.tile(np.array([+1], dtype=int), n_molecules),
            np.tile(np.array([0], dtype=int), n_solvent),
        ),
    )
    positions = np.zeros(shape=(n_particles, box_size.size), dtype=np.float32)
    head_positions = np.random.uniform(
            low=np.zeros_like(box_size),
            high=box_size,
            size=(n_molecules, box_size.size,),
        )
    for i in range(n_molecules):
        r0 = head_positions[i, :]
        positions[i * chain_length:(i + 1) * chain_length, :] = (
            R.random().apply(sds_single_positions) + r0[None, :]
        )
    counter_ions_positions = np.random.uniform(
        low=np.zeros_like(box_size),
        high=box_size,
        size=(n_molecules, box_size.size),
    )
    solvent_positions = np.random.uniform(
        low=np.zeros_like(box_size),
        high=box_size,
        size=(n_solvent, box_size.size),
    )
    positions[n_molecules * chain_length:n_molecules * (chain_length + 1), :] = (
        counter_ions_positions
    )
    positions[n_molecules * (chain_length + 1):, :] = solvent_positions

    with h5py.File(out_path, "w") as out_file:
        position_dataset = out_file.create_dataset(
            "coordinates",
            (1, n_particles, box_size.size,),
            dtype="float32",
        )
        types_dataset = out_file.create_dataset(
            "types",
            (n_particles,),
            dtype="i",
        )
        names_dataset = out_file.create_dataset(
            "names",
            (n_particles,),
            dtype="S10",
        )
        indices_dataset = out_file.create_dataset(
            "indices",
            (n_particles,),
            dtype="i",
        )
        charges_dataset = out_file.create_dataset(
            "charge",
            (n_particles,),
            dtype="float32",
        )
        molecules_dataset = out_file.create_dataset(
            "molecules",
            (n_particles,),
            dtype="i",
        )
        bonds_dataset = out_file.create_dataset(
            "bonds",
            (n_particles, 2,),
            dtype="i",
        )
        position_dataset[0, ...] = positions
        types_dataset[...] = types
        names_dataset[...] = names.astype(np.string_)
        indices_dataset[...] = np.arange(n_particles)
        molecules_dataset[...] = molecules
        bonds_dataset[...] = bonds
        charges_dataset[...] = charges

# 2204 SDS and 2204 counter-ions and 68 250 water in a 
# 8.3nm x 8.3nm x 8.3nm simulation box.
setup_system([16.6, 16.6, 16.6], 2204, 68250, "sds.HDF5")

#### Run the simulation
We the simulation:
```bash
mpirun -n 6 python3 ${HYMD}      \
                    sds.toml     \
                    sds.HDF5     \
                    --seed 1     \
                    --verbose 2
```